In [1]:
! pip install requests
import requests


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import requests

url = 'https://youtube.com/watch?v=9bZkp7q19f0'  # Example URL, replace with the desired one
response = requests.get(url)
# Check if the request was successful
print("successful" if response.status_code == 200 else "Page not found" if response.status_code == 404 else "Something went wrong")

successful
